# Check Databricks Connect

In [1]:
from databricks.connect import DatabricksSession
spark = DatabricksSession.builder.getOrCreate()

In [2]:
df = spark.table('fins_genai.agents.customer_profiles')
df.show(5)

+-------------+-------+----------+--------------+------------+---------------+---------+--------+----------+----------+------------------+----------+---------+--------------------+--------------+
|policy_number|CUST_ID|POLICYTYPE|pol_issue_date|pol_eff_date|pol_expiry_date|     MAKE|   MODEL|MODEL_YEAR|   DRV_DOB|           address|first_name|last_name|               email|  phone_number|
+-------------+-------+----------+--------------+------------+---------------+---------+--------+----------+----------+------------------+----------+---------+--------------------+--------------+
|    102147963| 7957.0|      COMP|    2020-07-29|  2020-07-29|     2021-08-28| MERCEDES|    C200|    2016.0|02-04-2013|   QUEENS, 11355.0|   Ricardo|   Rogers| hartjoe@example.org|(802)-895-4032|
|    102147937| 3304.0|      COMP|    2020-07-29|  2020-07-29|     2021-08-28|     AUDI|     A 3|    2019.0|30-10-2018|MANHATTAN, 10013.0|      Dawn|     Neal|  ssimon@example.org|(875)-209-8555|
|    102148003| 5173

# Set up Env and MLFlow

In [4]:
import mlflow
from dotenv import load_dotenv
load_dotenv('../.env')

DATABRICKS_HOST = os.getenv('host')
DATABRICKS_TOKEN = os.getenv('token')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
print(f"host: {DATABRICKS_HOST}")

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_registry_uri("http://localhost:5000")
mlflow.set_experiment("crewai_local_dev")
mlflow.crewai.autolog()

host: adb-984752964297111.11.azuredatabricks.net


2025/03/03 21:44:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed"


In [5]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()
set_uc_function_client(client)

CATALOG = 'fins_genai'
SCHEMA = 'agents'

# Create UC Tools

In [16]:
sql_function1 = \
f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.get_customer_transcripts_by_phone_number(
  search_name STRING
)
RETURNS TABLE (
  profile STRING
)
COMMENT 'Return a customer transcript with a given caller_id (phone number)'
LANGUAGE SQL
  RETURN
  select
    transcript
  from {CATALOG}.{SCHEMA}.call_transcripts
  where phone_number = search_name
;
"""
sql_func_info = client.create_function(sql_function_body=sql_function1)
print(sql_func_info)

FunctionInfo(browse_only=None, catalog_name='fins_genai', comment='Return a customer transcript with a given caller_id (phone number)', created_at=1741048361594, created_by='q.yu@databricks.com', data_type=<ColumnTypeName.TABLE_TYPE: 'TABLE_TYPE'>, external_language=None, external_name=None, full_data_type='(profile STRING)', full_name='fins_genai.agents.get_customer_transcripts_by_phone_number', function_id='0548fbb5-b5b6-4654-815e-11f11a41c345', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='search_name', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment=None, parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"search_name","type":"string","nullable":true,"metadata":{}}', type_precision=0, type_scale=0)]), is_deterministic=True, is_null_call=None, metastore_id='b86c6879-8c55-4e70-a585-18d16a4fa6e9', name='get_customer_transcripts

/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/unitycatalog/ai/core/databricks.py:308: UserWarning: The following parameters do not have descriptions: search_name for the function fins_genai.agents.get_customer_transcripts_by_phone_number. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(created_function_info)


In [22]:
sql_function2 = \
f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.get_customer_profile_by_phone_number(
  search_name STRING
)
RETURNS TABLE (
  profile STRING
)
COMMENT 'Return a customer profile with a given phone number'
LANGUAGE SQL
  RETURN
  SELECT
    CONCAT("The customer ", first_name, " ", last_name, " lives in ", address, " with email address ",
           email, " and phone number", phone_number, " owns a ", make, " ", model_year, " ", model, " has a policy of ", policytype, " (policy number:", policy_number, ") issued on ", pol_issue_date, " and expires on ", pol_expiry_date)
  as profile
  FROM {CATALOG}.{SCHEMA}.customer_profiles
  where phone_number = search_name
;
"""
sql_func_info = client.create_function(sql_function_body=sql_function2)
print(sql_func_info)

FunctionInfo(browse_only=None, catalog_name='fins_genai', comment='Return a customer profile with a given phone number', created_at=1741048996779, created_by='q.yu@databricks.com', data_type=<ColumnTypeName.TABLE_TYPE: 'TABLE_TYPE'>, external_language=None, external_name=None, full_data_type='(profile STRING)', full_name='fins_genai.agents.get_customer_profile_by_phone_number', function_id='e1628c1d-a034-48a9-a01e-5b678656bac2', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='search_name', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment=None, parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"search_name","type":"string","nullable":true,"metadata":{}}', type_precision=0, type_scale=0)]), is_deterministic=True, is_null_call=None, metastore_id='b86c6879-8c55-4e70-a585-18d16a4fa6e9', name='get_customer_profile_by_phone_number', owne

/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/unitycatalog/ai/core/databricks.py:308: UserWarning: The following parameters do not have descriptions: search_name for the function fins_genai.agents.get_customer_profile_by_phone_number. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(created_function_info)


In [6]:
sql_function3 = \
f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.search_policy(
    query STRING
)
RETURNS TABLE (
    policy STRING
)
COMMENT 'Return chunks from policy document for a given query'
LANGUAGE SQL
  RETURN
  SELECT
   chunked_text as policy
  FROM vector_search(
    index => 'fins_genai.agents.policy_docs_chunked_files_vs_index',
    query_text => query,
    num_results => 3
  )
;
"""
sql_func_info = client.create_function(sql_function_body=sql_function3)
print(sql_func_info)


FunctionInfo(browse_only=None, catalog_name='fins_genai', comment='Return chunks from policy document for a given query', created_at=1741063785744, created_by='q.yu@databricks.com', data_type=<ColumnTypeName.TABLE_TYPE: 'TABLE_TYPE'>, external_language=None, external_name=None, full_data_type='(policy STRING)', full_name='fins_genai.agents.search_policy', function_id='b38022cb-893d-4212-a5f7-4dec93e6dad4', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='query', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment=None, parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"query","type":"string","nullable":true,"metadata":{}}', type_precision=0, type_scale=0)]), is_deterministic=True, is_null_call=None, metastore_id='b86c6879-8c55-4e70-a585-18d16a4fa6e9', name='search_policy', owner='q.yu@databricks.com', parameter_style=<FunctionInfoPara

# Enable UC Tools

In [6]:
from unitycatalog.ai.crewai.toolkit import UCFunctionToolkit

functions = [f"{CATALOG}.{SCHEMA}.{func.name}" for func in client.list_functions(catalog=CATALOG,
                                                                                 schema=SCHEMA)
             if func.name.endswith('phone_number') or func.name == 'search_policy']
# Pass the UC function name that we created to the constructor
toolkit = UCFunctionToolkit(function_names=functions)

# Get the CrewAI-compatible tools definitions
tools = toolkit.tools

In [12]:
tools[0].fn(**{'search_name': '(875)-209-8555'})

/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/unitycatalog/ai/core/databricks.py:680: UserWarning: The following parameters do not have descriptions: search_name for the function fins_genai.agents.get_customer_profile_by_phone_number. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(function_info)


'{"format": "CSV", "value": "profile\\n\\"The customer Dawn Neal lives in MANHATTAN, 10013.0 with email address ssimon@example.org and phone number(875)-209-8555 owns a AUDI 2019.0 A 3 has a policy of COMP (policy number:102147937) issued on 2020-07-29 and expires on 2021-08-28\\"\\n", "truncated": false}'

In [13]:
tools[2].fn(**{'query': 'change car insurance policy'})

/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/unitycatalog/ai/core/databricks.py:680: UserWarning: The following parameters do not have descriptions: query for the function fins_genai.agents.search_policy. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(function_info)


'{"format": "CSV", "value": "policy\\n\\"ACCIDENT, YOU MAY CHOOSE ANY SINGLE AUTO\\nSHOWN ONTHEPOLICY DECLARATIONS AND THE\\nCOVERAGE LIMITS APPLICABLE TOTHAT AUTO\\nWILL APPLY.\\nTHELIMITS AVAILABLE FOR ANY OTHER AUTO\\nCOVERED BYTHEPOLICY WILL NOT BEADDED TO\\nTHECOVERAGE FOR THEINVOLVED ORCHOSEN\\nAUTO.\\nTransfer\\nThispolicy can\'t betransferred toanyone without our\\nwritten consent. However, ifyoudie,coverage will\\nbeprovided until theendofthepolicy period for:\\n1.your legal representative while acting assuch,\\nand\\n2.persons covered onthedate ofyour death.\\nCancellation\\nYoumay cancel thispolicy bywriting usthefuture\\ndate youwish tostop coverage.\\nAllstate may cancel partorallofthispolicy by\\nmailing notice toyouatyour lastknown address. If\\nwecancel because youdidn\'t paythepremium, the\\ndate ofcancellation willbeatleast 10days after the\\ndate ofmailing. Ifwecancel foranyreason other\\nPage 2\\nS\\nA\\nM\\nP\\nL\\nE\\nD\\nO\\nC\\nU\\nM\\nE\\nN\\nTAllstate Insuranc

# Create CrewAI Agents

In [9]:
import os
from crewai import Agent, Task, Crew

cs_profile_tool, find_transcript_tool, search_policy_tool = tools

transcript_analyzer = Agent(
    role="Transcript Analyzer",
    goal="Analyze the customer transcripts and detect the customer intent, concerns, sentiment "
         "and questions based on his or her phone number: {phone_number}",
    backstory="The agent is responsible for analyzing the customer transcripts and identifying "
              "the customer intent, concerns, sentiment, and questions.",
    tools=[find_transcript_tool],
    verbose=True
)

customer_service = Agent(
    role="Customer Service",
    goal="Address customer intents, concerns, answer general questions on policy, and provide "
         "service to improve customer happiness",
    backstory="The agent is responsible for addressing customer intents, concerns, questions on "
              "policy and providing service to improve customer sentiment.",
    tools=[cs_profile_tool, search_policy_tool],
    verbose=True
)

analysis = Task(
    description="Given a customer based on his or her phone number: {phone_number} analyze the call transcripts"
                "carefully and detect the customer intent, concerns, and questions.",
    expected_output="A detailed analysis of customer intents, concerns, sentiment, and questions.",
    agent=transcript_analyzer
)

customer_service_task = Task(
    description="Address customer intents, concerns, answer any general questions on policy using"
                "the policy document, and provide service to improve customer sentiment "
                "based on the analysis of the call transcripts.",
    expected_output="An personalized and politely written response to the customer intents, "
                    "concerns, questions on policy.",
    agent=customer_service
)

crew = Crew(
    agents=[transcript_analyzer, customer_service],
    tasks=[analysis, customer_service_task],
    verbose=True,
    planning=True
)


In [10]:
with mlflow.start_run():
    inputs = {"phone_number": "(875)-209-8555"}
    result = crew.kickoff(inputs=inputs)

2025/03/03 21:54:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/pydantic/functional_validators.py:787: UserWarning: Pydantic serializer warnings:
  Expected `InstanceOf` but got `CrewAgentExecutor` with value `<crewai.agents.crew_agent...r object at 0x34089b550>` - serialized value may not be as expected"


 
[2025-03-03 21:54:04][INFO]: Planning the crew execution
# Agent: Transcript Analyzer
## Task: Given a customer based on his or her phone number: (875)-209-8555 analyze the call transcriptscarefully and detect the customer intent, concerns, and questions.1. Access the tool 'fins_genai__agents__get_customer_transcripts_by_phone_number'.
2. Input the customer phone number '(875)-209-8555' into the tool arguments to retrieve the corresponding call transcripts.
3. Once the transcripts are obtained, carefully read through each conversation. Look for key phrases that indicate the customer's intent, such as requests for information or assistance.
4. Identify any concerns expressed by the customer, including specific problems or dissatisfaction noted during the call. Make a list of these concerns for reference.
5. Analyze the tone of the customer's communication to determine the sentiment, whether it is positive, negative, or neutral.
6. Catalog any questions raised by the customer, ensuring

/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/unitycatalog/ai/core/databricks.py:680: UserWarning: The following parameters do not have descriptions: search_name for the function fins_genai.agents.get_customer_transcripts_by_phone_number. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(function_info)




# Agent: Transcript Analyzer
## Using tool: fins_genai__agents__get_customer_transcripts_by_phone_number
## Tool Input: 
"{\"search_name\": \"(875)-209-8555\"}"
## Tool Output: 
{"format": "CSV", "value": "profile\n\"I need to file a claim for fire damage in my kitchen from a cooking accident.,   Sorry to hear that, but let's get one thing straight: we don't pay for foolish mistakes. If you're dripped grease on the stove and set the kitchen on fire, that's not our problem. You're gonna need to prove this was some kind of freak accident before we pay up. What's the point of even having insurance if you're just gonna be reckless in the kitchen? Alright, let's get the details. What's the make and model of your vehicle? And what's your policy number?,    I understand your concerns, but I assure you, this was not a foolish mistake. I am a responsible homeowner and I take great care in my cooking. The fire was not caused by dripped grease, but rather a malfunction in my stove's electrical 

/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/unitycatalog/ai/core/databricks.py:680: UserWarning: The following parameters do not have descriptions: search_name for the function fins_genai.agents.get_customer_profile_by_phone_number. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(function_info)




# Agent: Customer Service
## Thought: I need to gather the customer's profile using the phone number provided to understand John's history and any previous interactions, which will help me tailor my response.
## Using tool: fins_genai__agents__get_customer_profile_by_phone_number
## Tool Input: 
"{\"search_name\": \"(875)-209-8555\"}"
## Tool Output: 
{"format": "CSV", "value": "profile\n\"The customer Dawn Neal lives in MANHATTAN, 10013.0 with email address ssimon@example.org and phone number(875)-209-8555 owns a AUDI 2019.0 A 3 has a policy of COMP (policy number:102147937) issued on 2020-07-29 and expires on 2021-08-28\"\n", "truncated": false}


/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/unitycatalog/ai/core/databricks.py:680: UserWarning: The following parameters do not have descriptions: query for the function fins_genai.agents.search_policy. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(function_info)




# Agent: Customer Service
## Thought: Thought: I have gathered the customer's profile, which includes essential details. Now, I will search the policy document for relevant information that addresses John's questions regarding the claims process and any other concerns raised.
## Using tool: fins_genai__agents__search_policy
## Tool Input: 
"{\"query\": \"steps to file a claim for fire damage and theft claim process requirements\"}"
## Tool Output: 
{"format": "CSV", "value": "policy\n\"ANY COVERAGE ZADEDUCTIBLE WILL ALWAYS\nAPPLY.Action Against Allstate\nNoonemay sueusunder thiscoverage unless there\nisfullcompliance with allthepolicy terms.\nSubrogation Rights\nWhen wepay,your rights ofrecovery from anyone\nelsebecome ours uptotheamount wehave paid.\nYoumust protect these rights andhelp usenforce\nthem.\nWhat YouMust DoIfThere IsALoss\n1.Assoon aspossible, anyperson making claim\nmust giveuswritten proof ofloss. Itmust\ninclude alldetails reasonably required byus.\nWehave theright t

Trace(request_id=98146d20c05d4607b6ccfc2eb7f20413)

In [15]:
print(result.raw)

Dear John,

Thank you for reaching out to us regarding your claims for fire damage and theft. We understand how important these matters are to you and appreciate your patience as we address your inquiries.

To file your claim for the fire damage, please follow these steps:

1. As soon as possible, provide us with written proof of loss. This should include all details reasonably required by us.
2. We have the right to inspect the damaged property and may require you to file a sworn proof of loss.
3. It's essential to protect the property from further damage, and we will cover reasonable expenses incurred to guard against additional loss.
4. If there was a theft, please report it promptly to the police.

Regarding your concern about our initial judgment on the kitchen fire accident, please know that your long-term status as a valued customer is important to us. We take every claim seriously and will ensure that your claim is thoroughly reviewed.

For your theft claim, please ensure that 